In [1]:
# IMPORTS
import cvxpy as cp
import numpy as np
import numpy.linalg
import numpy.matlib
from numpy.linalg import norm
import algorithms.trend_filter_derivative_cvx as tfilter
from scipy import interpolate
from scipy import signal
import csv   
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# LOAD DATA

# 1) LOAD DYNAMICS AND TIME ARRAYS

filepath = 'data_files/Lorenz63_dynamics.csv'
dynamics = np.genfromtxt(filepath, delimiter=',')

t = dynamics[0]
data = dynamics[1:]
n_states = int(data.shape[0] / 2)
X = data[0:n_states]
DX = data[n_states:]
t_samples = len(t)

noise_lvls = [0.001]
n_noise_lvls = len(noise_lvls) 

failed_samples_state = []
failed_samples = []

for noise in range(n_noise_lvls):

    # 2) LOAD OBSERVATION SAMPLES
    Y_samples = []
    noise_lvl = str(noise_lvls[noise])

    for state in range(n_states):

        filepath = 'data_files/Lorenz63_samples_noise_' + noise_lvl +'_Y'+ str(state+1) + '.csv'
        y = np.genfromtxt(filepath, delimiter=',')

        Y_samples.append(y)

    n_samples = Y_samples[0].shape[0]

    ################### DENOISING AND NUMERICAL DIFFERENTIATION - SMOOTHING SPLINES ###################
    trim = int(np.floor(5 / 100 * t_samples))
    error_dX = np.zeros((n_samples,n_states))
    tt_samples = t_samples - 2 * trim
    DYhat_tf = np.zeros((n_samples, n_states, tt_samples))
    
    for state in range(n_states):

        for sample in range(n_samples):
        #CVX code
            y = Y_samples[state][sample,:]
            
            try:
                #Find corner point
                tf_lc = tfilter.lcurve_corner(y, t, order = 3, lambda_min = 1e-12, lambda_max = 1)
                dy_tf = tf_lc[0]

                #Remove ends
                DYhat_tf[sample,state,:] = dy_tf[trim:-trim]
                #Compute errors
                error_dX[sample,state] = norm(DYhat_tf[sample,state,:] - DX[state,trim:-trim]) / norm(DX[state,trim:-trim])
            
            except:
                print('Failed to converge. Ignoring current sample...')
                failed_samples_state.extend([sample])
                continue
                
        failed_samples.append(failed_samples_state)
        failed_samples_state = []   

  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 18 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 19 iterations.


  Convergence criterion reached in 17 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 16 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 19 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 18 iterations.
  Convergence criterion reached in 17 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 16 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 17 iterations.
  Convergence criterion reached in 21 iterations.
  Convergence criterion reached in 20 iterations.
  Convergence criterion reached in 18 iterations.
  Convergence criterion reached in 17 iterations.


In [3]:
DYhat_tf_mod = []
error_dX_mod = []
mean_dX_error = np.zeros((n_states))
var_dX_error = np.zeros((n_states))

#Remove failed samples
for state in range(n_states):
    DYhat_tf_mod.append(np.delete(DYhat_tf[:,state,:], failed_samples[state], axis = 0))
    error_dX_mod.append(np.delete(error_dX[:,state], failed_samples[state], axis = 0))
    mean_dX_error[state] = error_dX_mod[state].mean()
    var_dX_error[state] = error_dX_mod[state].var()
    
    filename_dy = 'data_files/trendfilter_diff/Lorenz63_trendfilter_diff_v2_' + noise_lvl + '_dY' + str(state+1) + '.csv'
    with open(filename_dy, 'w') as csvfile:   
        # creating a csv writer object   
        csvwriter = csv.writer(csvfile)   
        # writing the data rows   
        csvwriter.writerows(DYhat_tf_mod[state])


In [4]:

temp = [['$\sigma$ = ' + str(x)] * n_states for x in noise_lvls]
noise_indx = np.array([item for sublist in temp for item in sublist])

state_indx = np.array(['state x' + str(x+1) for x in range(n_states)] * n_noise_lvls)

arrays = [noise_indx, state_indx]
data = np.vstack((mean_dX_error.T.flatten(), var_dX_error.T.flatten())).T

col_names = ['mean error dx', 'var. error dx']
df = pd.DataFrame(data, index=arrays, columns=col_names)
df

mean error dx  var. error dx
$\sigma$ = 1 state x1       0.130768       0.000610
             state x2       0.126356       0.000354
             state x3       0.081832       0.000171

In [ ]:
#cHEAT SHEET

#lambda_min = 1e-8, lambda_max = 1000